# Library import

### import librerie

In [ ]:
import os
import numpy as np

import json
from PIL import Image
from torch.utils.data import Dataset

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.optim import Adam
from tqdm import tqdm

import ast

In [ ]:
pip install triton

## Path

In [ ]:
# file contenente i path delle immagini del dataset
txt_file = "/kaggle/input/our-xview-dataset/xView_class_map.json"
img_dir = "/kaggle/input/our-xview-dataset/images"

annotation_file = "/kaggle/input/our-xview-dataset/COCO_annotations_new.json"

In [ ]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils', trust_repo=True)

# Dataloader

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

def visualize_dataset_element(dataset, idx):
    # Estrai l'elemento dal dataset
    image_tensor, boxes_tensor, labels_tensor = dataset[idx]
    
    # Converti il tensore dell'immagine in un formato che matplotlib può visualizzare
    # Il tensore è in formato (C, H, W), quindi dobbiamo convertirlo in (H, W, C)
    image = image_tensor.permute(1, 2, 0).numpy()
    
    # Rescale l'immagine (perché il modello SSD può usare valori normalizzati tra [0, 1])
    image = np.clip(image, 0, 1)

    # Crea una figura per visualizzare l'immagine
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    # Estrai i bounding box e le etichette
    boxes = boxes_tensor.numpy()
    labels = labels_tensor.numpy()

    # Visualizza i bounding box
    for box, label in zip(boxes, labels):
        # I bounding box sono [xmin, ymin, xmax, ymax]
        xmin, ymin, xmax, ymax = box
        width = xmax - xmin
        height = ymax - ymin
        
        # Aggiungi un rettangolo per ogni bounding box
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Aggiungi l'etichetta del bounding box
        ax.text(xmin, ymin, f'Class {int(label)}', color='yellow', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))
    
    # Mostra l'immagine con i bounding box
    plt.show()


In [ ]:
# Define transformations
from torchvision.transforms import Compose, ToTensor

# Dataset and Dataloader
train_dataset = SSDDataset(image_dir, annotations_file, transform=ToTensor())
#val_dataset = CustomDataset(annotation_file, img_dir, transforms=train_transforms)

train_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=utils.collate_fn)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_dataset_element(dataset, idx):
    """
    Visualizza un elemento specifico del dataset (immagine, bounding box e label).
    
    Args:
        dataset: Istanza del dataset.
        idx (int): Indice dell'elemento da visualizzare.
    """
    # Estrai un elemento dal dataset
    element = dataset[idx]
    image = element['image']
    boxes = element['boxes']
    labels = element['labels']

    print(f"boxes : {boxes}")
    print(f"labels : {labels}")
    
    # Converti l'immagine per la visualizzazione
    image = image.permute(1, 2, 0).numpy()  # Da [C, H, W] a [H, W, C]
    image = (image * 0.229 + 0.485).clip(0, 1)  # De-normalizzazione (assume mean/std standard)

    # Visualizza l'immagine
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    # Aggiungi i bounding box
    for bbox, label in zip(boxes, labels):
        x_min, y_min, x_max, y_max = bbox.numpy()
        rect = patches.Rectangle(
            (x_min, y_min), x_max - x_min, y_max - y_min,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(
            x_min, y_min - 5, f"Label: {label.item()}",
            color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5)
        )

    plt.axis('off')
    plt.show()

# Esempio di utilizzo
visualize_dataset_element(dataset, idx=5)  # Visualizza l'elemento con indice 5 del dataset


# Network

In [ ]:
class SSDModel(nn.Module):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()

        self.num_classes = num_classes
        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd') # modello pre-addestrato su dataset COCO

        # FAI L'ANALISI DOPO
        # Modify the last layers (loc and conf) to handle 'num_classes'
        for i in range(len(self.model.loc)):
            in_channels = self.model.loc[i].in_channels
            # Modify the location (bbox regression) layer
            self.model.loc[i] = nn.Conv2d(in_channels, 4, kernel_size=3, padding=1)
        
        for i in range(len(self.model.conf)):
            in_channels = self.model.conf[i].in_channels
            # Modify the classification layer to output 'num_classes + 1' (for background class)
            self.model.conf[i] = nn.Conv2d(in_channels, self.num_classes, kernel_size=3, padding=1)


    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.ssd_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [ ]:
# Initialize model and trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SSDModel(num_classes=12).to(device)

print(model)

# Training

In [ ]:
import torch
import time

class Trainer:
    def __init__(self, model, dataloader, optimizer, loss_fn, device, num_epochs=10, save_path=None):
        self.model = model
        self.dataloader = dataloader
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.device = device
        self.num_epochs = num_epochs
        self.save_path = save_path

    def train_one_epoch(self):
        self.model.train()
        running_loss = 0.0

        for images, boxes, labels in self.dataloader:
            # Assicurati che i dati siano sulla stessa device (CPU o GPU)
            images = images.to(self.device)
            boxes = boxes.to(self.device)
            labels = labels.to(self.device)

            # Inizializza il gradiente
            self.optimizer.zero_grad()

            # Fai una previsione
            outputs = self.model(images)

            # Calcola la perdita
            loss = self.loss_fn(outputs, boxes, labels)
            running_loss += loss.item()

            # Backpropagation e aggiornamento dei pesi
            loss.backward()
            self.optimizer.step()

        # Calcola la perdita media per l'epoca
        avg_loss = running_loss / len(self.dataloader)
        return avg_loss

    def train(self):
        for epoch in range(self.num_epochs):
            start_time = time.time()
            avg_loss = self.train_one_epoch()

            elapsed_time = time.time() - start_time
            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {avg_loss:.4f}, Time: {elapsed_time:.2f} seconds")

            # Salvataggio del modello (opzionale)
            if self.save_path:
                torch.save(self.model.state_dict(), self.save_path)

    def evaluate(self, dataloader):
        self.model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for images, boxes, labels in dataloader:
                # Assicurati che i dati siano sulla stessa device (CPU o GPU)
                images = images.to(self.device)
                boxes = boxes.to(self.device)
                labels = labels.to(self.device)

                # Fai una previsione
                outputs = self.model(images)

                # Calcola la perdita
                loss = self.loss_fn(outputs, boxes, labels)
                running_loss += loss.item()

        # Calcola la perdita media
        avg_loss = running_loss / len(dataloader)
        return avg_loss


In [ ]:
# Crea l'ottimizzatore (ad esempio, Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Crea la funzione di perdita
# Supponiamo che `compute_loss` sia una funzione del modello che restituisce la perdita
def loss_fn(outputs, boxes, labels):
    return model.compute_loss(outputs, boxes, labels)

# Crea il dataloader (supponiamo di avere già il dataset)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=utils.collate_fn)

# Inizializza il trainer
trainer = Trainer(model, dataloader, optimizer, loss_fn, device, num_epochs=10, save_path="ssd_model.pth")

# Avvia l'addestramento
trainer.train()

# Testing